# Setting

In [1]:
import glob
import json
import os
import numpy as np
import pickle
from PIL import Image
import shutil
from tqdm.autonotebook import tqdm

/tmp/ipykernel_27347/3993891805.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Split File

In [2]:
datadirbase = '/workspace/data/Train'
savedirbase = '/workspace/data/trainV1'

In [19]:
video_keys = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(datadirbase, '*')))]

In [20]:
valid_video_keys = [
    'IR_18',
    'IR_25',
    'THERM_32',

    'THERM_11',
    'IR_3',
    'IR_5',
    'THERM_9',

    'THERM_40',
    'THERM_50',
    'THERM_44',
]
train_video_keys = [x for x in video_keys if not x in valid_video_keys]

In [21]:
savedirX = os.path.join(savedirbase, 'train')
os.makedirs(savedirX, exist_ok=True)
for k in tqdm(train_video_keys):
    imgfiles = sorted(glob.glob(os.path.join(datadirbase, k, 'img1/*.jpg')))
    
    for x in imgfiles[::30]:
        basefn = os.path.basename(x)
        newfilename = f'{k}_{basefn}'
        
        shutil.copy(x, os.path.join(savedirX, newfilename))

  0%|          | 0/47 [00:00<?, ?it/s]

In [22]:
savedirX = os.path.join(savedirbase, 'valid')
os.makedirs(savedirX, exist_ok=True)
for k in tqdm(valid_video_keys):
    imgfiles = sorted(glob.glob(os.path.join(datadirbase, k, 'img1/*.jpg')))
    
    for x in imgfiles[::30]:
        basefn = os.path.basename(x)
        newfilename = f'{k}_{basefn}'
        
        shutil.copy(x, os.path.join(savedirX, newfilename))

  0%|          | 0/10 [00:00<?, ?it/s]

# To COCO

In [50]:
datadirbase = '/workspace/data/Train'
savedirbase = '/workspace/data/trainV1'
gtfiles = sorted(glob.glob(os.path.join(datadirbase, '*/gt/*.txt')))

imgfiles = sorted(glob.glob(os.path.join(savedirbase, 'valid/*.jpg')))

In [51]:
dsannot = {
    'info': {},
    'license':[{'id': 1, 'url': '', 'name': 'Unknown'}],
    'categories': [
        {'id': 0, 'name': 'person', 'supercategory': 'none'},
    ],
}

In [52]:
dsannot_images = []
cnt = 0
for x in tqdm(imgfiles):
    im = Image.open(x)
    w, h = im.size
    dsannot_images.append({
        'id': cnt,
        'license': 1,
        'file_name': os.path.basename(x),
        'height': h,
        'width': w,
    })
    cnt += 1
dsannot['images'] = dsannot_images

  0%|          | 0/107 [00:00<?, ?it/s]

In [53]:
def find_image_id(key, images, extension='.jpg'):
    for x in images:
        if x['file_name'].split(extension)[0] == key:
            return x['id']
    return None

def find_cat_id(name, categories):
    for x in categories:
        if x['name'] == name:
            return x['id']
    return None

def find_cat_name(idx, categories):
    for x in categories:
        if x['id'] == idx:
            return x['name']
    return None

def parse_gt(framenumber, gts):
    for g in gts:
        frame, iid, bb_left, bb_top, bb_width, bb_height, conf, iclass, visibility = g.split(',')
        if int(frame) == int(framenumber):
            return float(bb_left), float(bb_top), float(bb_width), float(bb_height)

    return None

In [54]:
cnt = 0
annots = []
for x in tqdm(imgfiles):
    fn = os.path.basename(x)
    key = fn.split('.jpg')[0]
    image_id = find_image_id(key, dsannot['images'])
    if image_id is None:
        print(f'cannot find image_id for {k}')
        continue
        
        
    datatype, foldernumber, framenumber = key.split('_')
    foldername = '_'.join([datatype, foldernumber])
    gtfilepath = os.path.join(datadirbase, foldername, 'gt/gt.txt')
    gts = []
    with open(gtfilepath, 'r') as f:
        for line in f.readlines():
            gts.append(line.strip())        
            
    try:
        bb_left, bb_top, bb_width, bb_height = parse_gt(framenumber, gts)

        cannot = {
            'id': cnt,
            'image_id': image_id,
            'category_id': 0,
            'bbox': [bb_left, bb_top, bb_width, bb_height],
            'area': bb_width*bb_height,
            'segmentation': [],
            'iscrowd': 0
        }
        annots.append(cannot)

        cnt += 1 
    except:
        print(f'{x} is wrong')
            
dsannot['annotations'] = annots        

  0%|          | 0/107 [00:00<?, ?it/s]

In [55]:
with open(os.path.join(savedirbase, 'valid_annot.json'), 'w') as f:
    json.dump(dsannot, f)

In [ ]:
savedirbase = '/workspace/data/trainV1'

In [53]:
datatype, foldernumber, framenumber = key.split('_')

In [54]:
foldername = '_'.join([datatype, foldnumber])

In [60]:
gtfilepath = os.path.join(datadirbase, foldername, 'gt/gt.txt')

In [61]:
gts = []
with open(gtfilepath, 'r') as f:
    for line in f.readlines():
        gts.append(line.strip())

In [68]:
bb_left

358.0

In [64]:
frame

'1'

In [ ]:
annots = []
cnt = 0
for k in tqdm(keys):
    annotfile = os.path.join(annotbase, f'{k}.xml')
    tree = ET.parse(annotfile)
    root = tree.getroot()
    
    image_id = find_image_id(k, dsannot['images'])
    if image_id is None:
        print(f'cannot find image_id for {k}')
        continue
    
    for x in root.iter('object'):
        
        cat_name = None
        xmin = None
        ymin = None
        xmax = None
        ymax = None
        for c in x:
            if c.tag == 'name':
                cat_name = c.text
            if c.tag == 'bndbox':
                for m in c:
                    if m.tag == 'xmin':
                        xmin = float(m.text)
                    elif m.tag == 'xmax':
                        xmax = float(m.text)
                    elif m.tag == 'ymin':
                        ymin = float(m.text)
                    elif m.tag == 'ymax':
                        ymax = float(m.text)
        
        cat_id = find_cat_id(cat_name, dsannot['categories'])
        if cat_id is None:
            print(f'cannot find cat_id for {cat_name}')
            continue
            
        cannot = {
            'id': cnt,
            'image_id': image_id,
            'category_id': cat_id,
            'bbox': [xmin, ymin, xmax-xmin, ymax-ymin],
            'area': (xmax-xmin)*(ymax-ymin),
            'segmentation': [],
            'iscrowd': 0
        }
        annots.append(cannot)
        
        cnt += 1 
dsannot['annotations'] = annots

In [ ]:
 - <frame>, <id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <conf>, <class>, <visibility>

In [57]:
gtfiles[0]

'/workspace/data/Train/IR_1/gt/gt.txt'

In [58]:
datas = []
with open(gtfiles[0], 'r') as f:
    for line in f.readlines():
        datas.append(line.strip())

In [59]:
datas

['1,1,1108,316,98,368,1,1,1',
 '2,1,1107,317,99,366,1,1,1',
 '3,1,1107,317,99,366,1,1,1',
 '4,1,1107,318,99,364,1,1,1',
 '5,1,1107,322,99,360,1,1,1',
 '6,1,1107,330,99,352,1,1,1',
 '7,1,1107,333,100,349,1,1,1',
 '8,1,1107,337,100,345,1,1,1',
 '9,1,1110,342,100,340,1,1,1',
 '10,1,1110,356,100,326,1,1,1',
 '11,1,1113,358,100,326,1,1,1',
 '12,1,1111,375,96,305,1,1,1',
 '13,1,1111,375,110,305,1,1,1',
 '14,1,1111,375,106,305,1,1,1',
 '15,1,1109,381,111,298,1,1,1',
 '16,1,1109,381,109,298,1,1,1',
 '17,1,1111,380,107,299,1,1,1',
 '18,1,1112,379,105,301,1,1,1',
 '19,1,1114,379,103,301,1,1,1',
 '20,1,1119,376,98,304,1,1,1',
 '21,1,1120,375,96,309,1,1,1',
 '22,1,1120,375,94,311,1,1,1',
 '23,1,1117,376,94,311,1,1,1',
 '24,1,1114,376,94,311,1,1,1',
 '25,1,1112,380,93,311,1,1,1',
 '26,1,1106,380,95,313,1,1,1',
 '27,1,1096,382,105,313,1,1,1',
 '28,1,1087,382,109,312,1,1,1',
 '29,1,1079,385,116,309,1,1,1',
 '30,1,1072,385,117,309,1,1,1',
 '31,1,1067,387,118,307,1,1,1',
 '32,1,1062,391,119,302,1,1,1',